In [1]:
import re
import typing as t
from collections import defaultdict
from functools import lru_cache
from pathlib import Path

import nltk
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from nltk.corpus import stopwords, wordnet
from sklearn import metrics
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader, Subset, random_split

In [2]:
def common_train(
        model: nn.Module,
        loss_fn: nn.Module,
        optimizer: optim.Optimizer,
        train_dataloader: DataLoader,
        epochs: int,
        test_dataloader: DataLoader = None,
        lr_scheduler=None,
        verbose: int = 100,
        device: str = "mps",
) -> t.List[float]:
    """
    Функция common_train() обучает модель nn.Module с использованием функции потерь nn.Module,
    оптимизатора optim.Optimizer и даталоадера для обучающего набора данных train_dataloader на
    заданное число эпох epochs.
    
    Опционально, может быть передан также даталоадер для тестового набора данных test_dataloader и шедулер
    обучения с убыванием скорости обучения lr_scheduler.
    Функция возвращает список значений функции потерь на каждой эпохе.
    """
    train_losses = []
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}\n" + "-" * 32)
        train_loss = train_loop(
            train_dataloader,
            model,
            loss_fn,
            optimizer,
            verbose=verbose,
            device=device,
        )
        train_losses.append(train_loss.item())
        if test_dataloader:
            loss, acc = test_loop(test_dataloader, model, loss_fn, device=device)
            if lr_scheduler:
                lr_scheduler.step(loss)
        torch.cuda.empty_cache()
    return train_losses


def train_loop(
        dataloader: DataLoader,
        model: nn.Module,
        loss_fn: nn.Module,
        optimizer: optim.Optimizer,
        verbose: int = 100,
        device: str = "cpu",
) -> torch.Tensor:
    """
    Функция train_loop() выполняет обучение модели на одной эпохе с использованием
    даталоадера обучающего набора данных dataloader, модели nn.Module,
    функции потерь nn.Module и оптимизатора optim.Optimizer.
    """
    model.train()

    size = len(dataloader.dataset)  # noqa
    num_batches = len(dataloader)
    avg_loss = 0

    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)

        pred = model(x)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward(retain_graph=True)
        optimizer.step()

        avg_loss += loss
        if batch % verbose == 0:
            print(f"loss: {loss:>7f}  [{batch * len(x):>5d}/{size:>5d}]")

        del x, y, pred, loss
        torch.cuda.empty_cache()

    return avg_loss / num_batches


@torch.no_grad()
def test_loop(
        dataloader: DataLoader,
        model: nn.Module,
        loss_fn: nn.Module,
        device: str = "cpu",
) -> t.Tuple[torch.Tensor, torch.Tensor]:
    """
    Функция test_loop() выполняет тестирование модели с использованием
    даталоадера тестового набора данных dataloader,
    модели nn.Module и функции потерь nn.Module. Функция 
    возвращает среднее значение функции потерь и точность на тестовом наборе данных.
    """
    model.eval()

    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    avg_loss, correct = 0, 0

    for x, y in dataloader:
        x, y = x.to(device), y.to(device)
        pred = model(x)
        avg_loss += loss_fn(pred, y)
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()  # noqa

        del x, y, pred
        torch.cuda.empty_cache()

    avg_loss /= num_batches
    accuracy = correct / size
    print(f"Test Error: \n Accuracy: {accuracy:>4f}, Avg loss: {avg_loss:>8f} \n")

    return avg_loss, accuracy


def train_test_split(dataset: t.Union[Dataset, t.Sized], train_part: float) -> t.Tuple[Subset, Subset]:
    """
    Функция train_test_split() разбивает набор данных dataset на обучающий и
    тестовый наборы с использованием заданной доли обучающего набора train_part.
    Функция возвращает кортеж, содержащий обучающий и тестовый наборы данных.
    """
    train_size = round(train_part * len(dataset))
    test_size = len(dataset) - train_size
    train_dataset, test_dataset = random_split(dataset, lengths=(train_size, test_size))
    return train_dataset, test_dataset


@torch.no_grad()
def get_y_test_y_pred(
        model: nn.Module,
        test_dataloader: DataLoader,
        device: str = "mps",
) -> t.Tuple[torch.Tensor, torch.Tensor]:
    model.eval()

    y_test = []
    y_pred = []
    for x, y in test_dataloader:
        x, y = x.to(device), y.to(device)
        pred = model(x).argmax(1)
        y_test.append(y)
        y_pred.append(pred)

        del x
        torch.cuda.empty_cache()

    return torch.hstack(y_test).detach().cpu(), torch.hstack(y_pred).detach().cpu()

## 1. Классификация фамилий (RNN)

Датасет: https://disk.yandex.ru/d/frNchuaBQVLxyA?w=1

1.1 Используя класс `nn.RNNCell` (абстракцию для отдельного временного шага RNN), реализуйте простейшую рекуррентную сеть Элмана в виде класса `RNN`. Используя созданный класс `RNN`, решите задачу классификации фамилий. 


In [3]:
class RNN(nn.Module):
    
    """
    Этот класс реализует обычную рекуррентную сеть (RNN) с использованием стандартного модуля nn.RNNCell из PyTorch.
    В конструкторе класса задается размер входных данных (input_size) и размер скрытого состояния (hidden_size),
    а также создается экземпляр nn.RNNCell с указанными размерами входных данных и скрытого состояния.
    """

    def __init__(self, input_size: int, hidden_size: int):
        super().__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.rnn_cell = nn.RNNCell(input_size, hidden_size)

    def forward(self, inputs: torch.Tensor, hx: torch.Tensor = None):
        """
        В методе forward реализуется сама RNN.
        Входные данные (inputs) имеют размерность batch_size x sequence_size x input_size,
        что означает, что в одном батче представлено sequence_size штук входных данных размера input_size.
        Переменная hx содержит начальное состояние скрытого слоя (если не задано, то создается тензор нулей размера batch_size x hidden_size).
        Также данные inputs транспонируются, чтобы в одной итерации цикла обрабатывался один такт времени.
        """
        batch_size, sequence_size, _ = inputs.size()
        inputs = inputs.permute(1, 0, 2)

        if hx is None:
            hx = torch.zeros(batch_size, self.hidden_size, dtype=inputs.dtype, device=inputs.device)
        else:
            hx = hx.squeeze(0)

        hidden = []
        for i in range(sequence_size):
            hx = self.rnn_cell(inputs[i], hx)
            hidden.append(hx)

        hidden = torch.stack(hidden)
        hx = hidden[-1].unsqueeze(0)
        return hidden.permute(1, 0, 2), hx

RNN check

In [4]:
"""
Cоздаются входные данные (inputs) и начальное скрытое состояние (hx) случайным образом с помощью функции torch.randn.
Далее создается экземпляр класса RNN с заданными размерами входных данных (input_size) и скрытого состояния (hidden_size).

Также создается экземпляр стандартного модуля nn.RNN с такими же размерами, но с параметром batch_first=True, что означает,
что размерность батча находится на первом месте (а не на втором, как у входных данных inputs).
"""

torch.manual_seed(0)

input_size, hidden_size = 4, 5
inputs = torch.randn(2, 3, input_size)
hx = torch.randn(1, 2, hidden_size)

torch.manual_seed(0)
my_rnn = RNN(input_size=input_size, hidden_size=hidden_size)

torch.manual_seed(0)
true_rnn = nn.RNN(input_size=input_size, hidden_size=hidden_size, batch_first=True)

# Функция рассчитывает скрытые состояния и выходные данные RNN для указанных входных данных и начального скрытого состояния.
my_rnn(inputs, hx)

(tensor([[[ 0.6515,  0.5430,  0.4023,  0.6325, -0.6068],
          [ 0.9149, -0.1088,  0.6385, -0.7387,  0.7532],
          [-0.6936,  0.5123, -0.2784, -0.5693, -0.0055]],
 
         [[ 0.1954,  0.6152,  0.2958, -0.8005,  0.8074],
          [-0.4577,  0.7566,  0.2972, -0.8834,  0.1265],
          [ 0.7166,  0.1516,  0.8047, -0.2007,  0.8192]]],
        grad_fn=<PermuteBackward0>),
 tensor([[[-0.6936,  0.5123, -0.2784, -0.5693, -0.0055],
          [ 0.7166,  0.1516,  0.8047, -0.2007,  0.8192]]],
        grad_fn=<UnsqueezeBackward0>))

In [5]:
true_rnn(inputs, hx)

(tensor([[[ 0.6515,  0.5430,  0.4023,  0.6325, -0.6068],
          [ 0.9149, -0.1088,  0.6385, -0.7387,  0.7532],
          [-0.6936,  0.5123, -0.2784, -0.5693, -0.0055]],
 
         [[ 0.1954,  0.6152,  0.2958, -0.8005,  0.8074],
          [-0.4577,  0.7566,  0.2972, -0.8834,  0.1265],
          [ 0.7166,  0.1516,  0.8047, -0.2007,  0.8192]]],
        grad_fn=<TransposeBackward1>),
 tensor([[[-0.6936,  0.5123, -0.2784, -0.5693, -0.0055],
          [ 0.7166,  0.1516,  0.8047, -0.2007,  0.8192]]],
        grad_fn=<StackBackward0>))

In [6]:
class SurnamesRNNClassifier(nn.Module):
    """
    Этот класс реализует классификатор фамилий с использованием рекуррентной сети (RNN).
    В конструкторе класса задается количество эмбеддингов (num_embeddings), размерность эмбеддинга (embedding_dim),
    размер скрытого состояния RNN (rnn_hidden_size), размерность вектора (vector_size) и количество классов (num_classes).

    В классе создается экземпляр слоя эмбеддинга (nn.Embedding) с указанными размерами и индексом для нулевых значений (padding_idx=0). 
    Также создается экземпляр класса RNN с указанными размерами эмбеддинга и скрытого состояния.
    
    Далее создается сеть классификации с последовательностью слоев: первый слой линейной трансформации (nn.Linear),
    следующий слой ReLU-нелинейности (nn.ReLU), слой Dropout (nn.Dropout) и последний слой линейной трансформации для
    получения выходных данных.
    
    В функции forward входные данные x преобразуются с помощью слоя эмбеддинга, затем передаются в рекуррентную сеть
    (self.rnn) с текущим скрытым состоянием (self.hx). Полученные скрытые состояния и выходные данные сворачиваются в один вектор
    с помощью функции torch.flatten, а затем передаются в сеть классификации.
    
    Функция forward возвращает выходные данные сети классификации.
    """
    
    def __init__(
            self,
            num_embeddings: int,
            embedding_dim: int,
            rnn_hidden_size: int,
            vector_size: int,
            num_classes: int,
    ):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=num_embeddings, embedding_dim=embedding_dim, padding_idx=0)
        self.hx = None
        self.rnn = RNN(input_size=embedding_dim, hidden_size=rnn_hidden_size)
        self.classifier = nn.Sequential(
            nn.Linear(rnn_hidden_size * vector_size, 256),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(256, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.embedding(x)
        x, hx = self.rnn(x, self.hx)
        x = torch.flatten(x, 1)
        return self.classifier(x)

In [7]:
class SurnamesVocab:
    """
    Класс SurnamesVocab представляет собой словарь для кодирования/декодирования фамилий.
    
    В конструкторе класса (__init__) передается список surnames, содержащий фамилии. Они приводятся к нижнему регистру,
    а затем уникальные символы в них добавляются в атрибут alphabet класса, а максимальная длина фамилий сохраняется в max_len.
    Также создается словарь ch2i, в котором каждому символу из alphabet соответствует его индекс.    
    """
    pad = ""

    def __init__(self, surnames: t.List[str]):
        uniques = set()
        max_len = 0
        for w in map(str.lower, surnames):
            uniques.update(w)
            max_len = max(len(w), max_len)

        self.alphabet = [self.pad, *uniques]
        self.max_len = max_len
        self.ch2i = {ch: i for i, ch in enumerate(self.alphabet)}

    def __len__(self):
        """
        Метод __len__ возвращает длину alphabet.
        """
        return len(self.alphabet)

    def encode(self, word: str) -> torch.Tensor:
        """
        Метод encode принимает фамилию word и возвращает ее кодированное представление в виде тензора типа long.
        Для этого сначала создается список indices, содержащий индексы символов фамилии в alphabet.
        Затем список indices дополняется символами-заполнителями (pad), чтобы добиться длины max_len.
        """
        indices = [self.ch2i[ch] for ch in word]
        indices += [self.ch2i[self.pad]] * (self.max_len - len(indices))
        return torch.tensor(indices, dtype=torch.long)

    def decode(self, indices: torch.Tensor) -> str:
        """
        Метод decode принимает тензор indices и возвращает фамилию, которую он кодирует.
        Сначала находится индексы символов-заполнителей в тензоре indices с помощью функции torch.nonzero.
        Если такие индексы есть, то indices усекается до первого такого индекса.
        Затем фамилия создается из символов, соответствующих индексам в indices, используя alphabet.
        """
        pad_indices = torch.nonzero(indices == self.ch2i[self.pad], as_tuple=True)[0]
        if len(pad_indices):
            indices = indices[:pad_indices[0]]
        return "".join(self.alphabet[i] for i in indices)

In [8]:
class SurnamesDataset(Dataset):
    """
    Этот код реализует класс SurnamesDataset, который наследует от Dataset из библиотеки PyTorch.
    Он предназначен для хранения данных из CSV-файла, который содержит фамилии и национальности соответствующих людей.
    """
    df: pd.DataFrame
    surnames: t.List[str]
    vocab: SurnamesVocab
    labeler: LabelEncoder
    data: torch.Tensor
    targets: torch.Tensor

    def __init__(self, path: Path):
        """
        В методе __init__ класса считывается CSV-файл, сохраняются фамилии в список self.surnames,
        создается экземпляр класса SurnamesVocab для кодирования/декодирования фамилий, размер тензора,
        который будет возвращать SurnamesVocab, сохраняется в size, и создается тензор data с кодированными фамилиями.
        Затем создается экземпляр LabelEncoder из библиотеки scikit-learn для кодирования национальностей в целочисленные значения,
        и сохраняется результат в тензор self.targets.
        """
        self.df = pd.read_csv(path)

        self.surnames = self.df["surname"].tolist()
        self.vocab = SurnamesVocab(self.surnames)
        size = self.vocab.encode(self.surnames[0].lower()).size()
        data = torch.vstack([self.vocab.encode(w.lower()) for w in self.surnames])
        self.data = data.view(len(self.surnames), *size)

        self.labeler = LabelEncoder()
        targets = self.labeler.fit_transform(self.df["nationality"])
        self.targets = torch.tensor(targets, dtype=torch.long)

    def __len__(self):
        """Метод __len__ возвращает количество фамилий в датасете"""
        return self.data.size(0)

    def __getitem__(self, idx):
        """
        Метод __getitem__ возвращает i-ую запись (кодированную фамилию и национальность) в виде кортежа.
        """
        return self.data[idx], self.targets[idx]

    def encode(self, word: str) -> torch.Tensor:
        """Метод encode кодирует фамилию в тензор, а метод decode декодирует тензор с индексами символов в строку."""
        return self.vocab.encode(word)

    def decode(self, indices: torch.Tensor) -> str:
        """Декодирует тензор с индексами символов в строку."""
        return self.vocab.decode(indices)

In [9]:
surnames_dataset = SurnamesDataset("./data/surnames.csv")
len(surnames_dataset)

10980

In [10]:
torch.manual_seed(0)

train_surnames_dataset, test_surnames_dataset = train_test_split(surnames_dataset, train_part=0.8)
print(len(train_surnames_dataset), len(test_surnames_dataset))

8784 2196


RNN

In [11]:
torch.manual_seed(0)

handmade_rnn_net = SurnamesRNNClassifier(
    num_embeddings=len(surnames_dataset.vocab),
    embedding_dim=100,
    rnn_hidden_size=64,
    vector_size=surnames_dataset.vocab.max_len,
    num_classes=len(surnames_dataset.labeler.classes_),
).to("mps")
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(handmade_rnn_net.parameters(), lr=0.0015)

train_dataloader = DataLoader(train_surnames_dataset, batch_size=128, shuffle=True)
test_dataloader = DataLoader(test_surnames_dataset, batch_size=512)

In [12]:
%%time

_ = common_train(
    epochs=15,
    model=handmade_rnn_net,
    loss_fn=loss_fn,
    optimizer=optimizer,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    verbose=50,
    device="mps",
)

Epoch 1
--------------------------------
loss: 2.897021  [    0/ 8784]
loss: 1.476651  [ 6400/ 8784]
Test Error: 
 Accuracy: 0.624317, Avg loss: 1.305790 

Epoch 2
--------------------------------
loss: 1.264221  [    0/ 8784]
loss: 1.249909  [ 6400/ 8784]
Test Error: 
 Accuracy: 0.695811, Avg loss: 1.065277 

Epoch 3
--------------------------------
loss: 0.938034  [    0/ 8784]
loss: 1.174979  [ 6400/ 8784]
Test Error: 
 Accuracy: 0.717668, Avg loss: 0.971192 

Epoch 4
--------------------------------
loss: 0.887203  [    0/ 8784]
loss: 0.913490  [ 6400/ 8784]
Test Error: 
 Accuracy: 0.730874, Avg loss: 0.904017 

Epoch 5
--------------------------------
loss: 0.603007  [    0/ 8784]
loss: 0.776868  [ 6400/ 8784]
Test Error: 
 Accuracy: 0.745902, Avg loss: 0.875295 

Epoch 6
--------------------------------
loss: 0.732690  [    0/ 8784]
loss: 0.600548  [ 6400/ 8784]
Test Error: 
 Accuracy: 0.754098, Avg loss: 0.859181 

Epoch 7
--------------------------------
loss: 0.696304  [    0/

In [13]:
y_test, y_pred = get_y_test_y_pred(handmade_rnn_net, test_dataloader, "mps")

print(metrics.classification_report(
    y_true=y_test,
    y_pred=y_pred,
    target_names=surnames_dataset.labeler.classes_,
    zero_division=True,
))

              precision    recall  f1-score   support

      Arabic       0.97      1.00      0.98       340
     Chinese       0.70      0.74      0.72        38
       Czech       0.55      0.34      0.42        96
       Dutch       0.52      0.43      0.47        51
     English       0.74      0.86      0.80       573
      French       0.21      0.08      0.11        39
      German       0.51      0.43      0.47       121
       Greek       0.62      0.74      0.68        34
       Irish       0.79      0.30      0.43        37
     Italian       0.76      0.75      0.75       128
    Japanese       0.83      0.92      0.87       156
      Korean       0.33      0.30      0.32        10
      Polish       0.56      0.58      0.57        26
  Portuguese       0.25      0.11      0.15         9
     Russian       0.86      0.88      0.87       458
    Scottish       0.00      0.00      0.00        17
     Spanish       0.44      0.48      0.46        50
  Vietnamese       1.00    

1.2 Замените модуль `RNN` из 1.1 на модули `nn.RNN`, `nn.LSTM` и `nn.GRU` (не забудьте указать аргумент `batch_first=True`). Сравните результаты работы.

In [14]:
class SurnamesAutobotRNNClassifier(nn.Module):

    def __init__(
            self,
            rnn_cls: t.Union[t.Type[nn.RNN], t.Type[nn.LSTM], t.Type[nn.GRU]],
            num_embeddings: int,
            embedding_dim: int,
            rnn_hidden_size: int,
            vector_size: int,
            num_classes: int,
    ):
        super().__init__()
        self.embedding = nn.Embedding(num_embeddings=num_embeddings, embedding_dim=embedding_dim, padding_idx=0)
        self.hx = None
        self.rnn = rnn_cls(input_size=embedding_dim, hidden_size=rnn_hidden_size)
        self.classifier = nn.Sequential(
            nn.Linear(rnn_hidden_size * vector_size, 256),
            nn.ReLU(),
            nn.Dropout(),
            nn.Linear(256, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.embedding(x)
        x, hx = self.rnn(x, self.hx)
        x = torch.flatten(x, 1)
        return self.classifier(x)

nn.RNN

In [15]:
torch.manual_seed(0)

rnn_net = SurnamesAutobotRNNClassifier(
    rnn_cls=nn.RNN,
    num_embeddings=len(surnames_dataset.vocab),
    embedding_dim=100,
    rnn_hidden_size=64,
    vector_size=surnames_dataset.vocab.max_len,
    num_classes=len(surnames_dataset.labeler.classes_),
).to("mps")
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(rnn_net.parameters(), lr=0.0015)

train_dataloader = DataLoader(train_surnames_dataset, batch_size=128, shuffle=True)
test_dataloader = DataLoader(test_surnames_dataset, batch_size=512)

In [16]:
%%time

_ = common_train(
    epochs=15,
    model=rnn_net,
    loss_fn=loss_fn,
    optimizer=optimizer,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    verbose=50,
    device="mps",
)

Epoch 1
--------------------------------
loss: 2.917136  [    0/ 8784]
loss: 1.642190  [ 6400/ 8784]
Test Error: 
 Accuracy: 0.574681, Avg loss: 1.475679 

Epoch 2
--------------------------------
loss: 1.348413  [    0/ 8784]
loss: 1.366360  [ 6400/ 8784]
Test Error: 
 Accuracy: 0.652095, Avg loss: 1.204014 

Epoch 3
--------------------------------
loss: 1.208325  [    0/ 8784]
loss: 1.341371  [ 6400/ 8784]
Test Error: 
 Accuracy: 0.677596, Avg loss: 1.105765 

Epoch 4
--------------------------------
loss: 0.950162  [    0/ 8784]
loss: 1.117557  [ 6400/ 8784]
Test Error: 
 Accuracy: 0.699909, Avg loss: 1.030863 

Epoch 5
--------------------------------
loss: 0.723195  [    0/ 8784]
loss: 1.033264  [ 6400/ 8784]
Test Error: 
 Accuracy: 0.712204, Avg loss: 0.989619 

Epoch 6
--------------------------------
loss: 0.875449  [    0/ 8784]
loss: 0.763817  [ 6400/ 8784]
Test Error: 
 Accuracy: 0.713115, Avg loss: 0.965103 

Epoch 7
--------------------------------
loss: 0.692767  [    0/

In [17]:
y_test, y_pred = get_y_test_y_pred(rnn_net, test_dataloader, "mps")

print(metrics.classification_report(
    y_true=y_test,
    y_pred=y_pred,
    target_names=surnames_dataset.labeler.classes_,
    zero_division=True,
))

              precision    recall  f1-score   support

      Arabic       0.95      1.00      0.97       340
     Chinese       0.72      0.74      0.73        38
       Czech       0.46      0.27      0.34        96
       Dutch       0.67      0.31      0.43        51
     English       0.67      0.85      0.75       573
      French       0.06      0.03      0.04        39
      German       0.53      0.36      0.43       121
       Greek       0.58      0.53      0.55        34
       Irish       0.69      0.24      0.36        37
     Italian       0.60      0.69      0.64       128
    Japanese       0.82      0.90      0.86       156
      Korean       0.25      0.30      0.27        10
      Polish       0.64      0.27      0.38        26
  Portuguese       0.00      0.00      0.00         9
     Russian       0.84      0.85      0.85       458
    Scottish       0.00      0.00      0.00        17
     Spanish       0.52      0.34      0.41        50
  Vietnamese       0.50    

nn.LSTM

In [18]:
torch.manual_seed(0)

lstm_net = SurnamesAutobotRNNClassifier(
    rnn_cls=nn.LSTM,
    num_embeddings=len(surnames_dataset.vocab),
    embedding_dim=100,
    rnn_hidden_size=64,
    vector_size=surnames_dataset.vocab.max_len,
    num_classes=len(surnames_dataset.labeler.classes_),
).to("mps")
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(lstm_net.parameters(), lr=0.0015)

train_dataloader = DataLoader(train_surnames_dataset, batch_size=128, shuffle=True)
test_dataloader = DataLoader(test_surnames_dataset, batch_size=512)

In [19]:
%%time

_ = common_train(
    epochs=15,
    model=lstm_net,
    loss_fn=loss_fn,
    optimizer=optimizer,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    verbose=50,
    device="mps",
)

Epoch 1
--------------------------------
loss: 2.903144  [    0/ 8784]
loss: 1.855295  [ 6400/ 8784]
Test Error: 
 Accuracy: 0.530510, Avg loss: 1.642389 

Epoch 2
--------------------------------
loss: 1.842572  [    0/ 8784]
loss: 1.509554  [ 6400/ 8784]
Test Error: 
 Accuracy: 0.611566, Avg loss: 1.375607 

Epoch 3
--------------------------------
loss: 1.320147  [    0/ 8784]
loss: 1.186791  [ 6400/ 8784]
Test Error: 
 Accuracy: 0.650729, Avg loss: 1.203663 

Epoch 4
--------------------------------
loss: 1.066090  [    0/ 8784]
loss: 1.112567  [ 6400/ 8784]
Test Error: 
 Accuracy: 0.677140, Avg loss: 1.111108 

Epoch 5
--------------------------------
loss: 1.030352  [    0/ 8784]
loss: 1.086272  [ 6400/ 8784]
Test Error: 
 Accuracy: 0.695811, Avg loss: 1.043684 

Epoch 6
--------------------------------
loss: 0.940045  [    0/ 8784]
loss: 0.881155  [ 6400/ 8784]
Test Error: 
 Accuracy: 0.713570, Avg loss: 0.987410 

Epoch 7
--------------------------------
loss: 0.813504  [    0/

In [20]:
y_test, y_pred = get_y_test_y_pred(lstm_net, test_dataloader, "mps")

print(metrics.classification_report(
    y_true=y_test,
    y_pred=y_pred,
    target_names=surnames_dataset.labeler.classes_,
    zero_division=True,
))

              precision    recall  f1-score   support

      Arabic       0.94      1.00      0.97       340
     Chinese       0.66      0.76      0.71        38
       Czech       0.62      0.16      0.25        96
       Dutch       0.78      0.35      0.49        51
     English       0.66      0.87      0.75       573
      French       0.00      0.00      0.00        39
      German       0.56      0.40      0.46       121
       Greek       0.67      0.59      0.62        34
       Irish       0.73      0.22      0.33        37
     Italian       0.57      0.72      0.64       128
    Japanese       0.81      0.84      0.83       156
      Korean       0.20      0.10      0.13        10
      Polish       0.48      0.46      0.47        26
  Portuguese       0.00      0.00      0.00         9
     Russian       0.85      0.85      0.85       458
    Scottish       0.00      0.00      0.00        17
     Spanish       0.52      0.32      0.40        50
  Vietnamese       1.00    

nn.GRU

In [21]:
torch.manual_seed(0)

gru_net = SurnamesAutobotRNNClassifier(
    rnn_cls=nn.GRU,
    num_embeddings=len(surnames_dataset.vocab),
    embedding_dim=100,
    rnn_hidden_size=64,
    vector_size=surnames_dataset.vocab.max_len,
    num_classes=len(surnames_dataset.labeler.classes_),
).to("mps")
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.Adam(gru_net.parameters(), lr=0.0015)

train_dataloader = DataLoader(train_surnames_dataset, batch_size=128, shuffle=True)
test_dataloader = DataLoader(test_surnames_dataset, batch_size=512)

In [22]:
%%time

_ = common_train(
    epochs=15,
    model=gru_net,
    loss_fn=loss_fn,
    optimizer=optimizer,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    verbose=50,
    device="mps",
)

Epoch 1
--------------------------------
loss:     inf  [    0/ 8784]
loss: 2.811437  [ 6400/ 8784]
Test Error: 
 Accuracy: 0.260929, Avg loss: 2.788394 

Epoch 2
--------------------------------
loss: 2.771024  [    0/ 8784]
loss: 2.729220  [ 6400/ 8784]
Test Error: 
 Accuracy: 0.260929, Avg loss: 2.709271 

Epoch 3
--------------------------------
loss: 2.710329  [    0/ 8784]
loss: 2.656490  [ 6400/ 8784]
Test Error: 
 Accuracy: 0.260929, Avg loss: 2.638715 

Epoch 4
--------------------------------
loss: 2.631357  [    0/ 8784]
loss: 2.600608  [ 6400/ 8784]
Test Error: 
 Accuracy: 0.260929, Avg loss: 2.577541 

Epoch 5
--------------------------------
loss: 2.534152  [    0/ 8784]
loss: 2.549181  [ 6400/ 8784]
Test Error: 
 Accuracy: 0.260929, Avg loss: 2.524147 

Epoch 6
--------------------------------
loss: 2.520626  [    0/ 8784]
loss: 2.505927  [ 6400/ 8784]
Test Error: 
 Accuracy: 0.260929, Avg loss: 2.478398 

Epoch 7
--------------------------------
loss: 2.481509  [    0/ 

In [23]:
y_test, y_pred = get_y_test_y_pred(gru_net, test_dataloader, "mps")

print(metrics.classification_report(
    y_true=y_test,
    y_pred=y_pred,
    target_names=surnames_dataset.labeler.classes_,
    zero_division=True,
))

              precision    recall  f1-score   support

      Arabic       1.00      0.00      0.00       340
     Chinese       1.00      0.00      0.00        38
       Czech       1.00      0.00      0.00        96
       Dutch       1.00      0.00      0.00        51
     English       0.26      1.00      0.41       573
      French       1.00      0.00      0.00        39
      German       1.00      0.00      0.00       121
       Greek       1.00      0.00      0.00        34
       Irish       1.00      0.00      0.00        37
     Italian       1.00      0.00      0.00       128
    Japanese       1.00      0.00      0.00       156
      Korean       1.00      0.00      0.00        10
      Polish       1.00      0.00      0.00        26
  Portuguese       1.00      0.00      0.00         9
     Russian       1.00      0.00      0.00       458
    Scottish       1.00      0.00      0.00        17
     Spanish       1.00      0.00      0.00        50
  Vietnamese       1.00    

nn.LSTM

nn.GRU

nn.RNN

Вывод: nn.LSTM предпочтительнее использовать